In [1]:
# Load data
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.preprocessing
from sklearn.linear_model import LogisticRegression
import project_env
from imp import reload
import math
import os
import run_logreg
import project_env
from sklearn.metrics import precision_recall_curve

#reload(project_env)
#reload(run_logreg)

%matplotlib inline


**The `project_env` package**

I wrote this python package so loading and working with data is quicker and easier. There are several convenience methods:
* `load_split_bucket(station_id)` - Load data for a bike station that's already pre-split into train, dev and test. Includes doing data cleaning and thresholding. The output is a dictionary:
```
  {
    'train': (DataFrame, Series),
    'dev': (DataFrame, Series),
    'test': (DataFrame, Series)
  }
```
  Each `(DataFrame, Series)` tuple is the feature values and target variables, respectively.
* `merge_training(split, df)` - Given two outputs of `load()`, append the training set of the second argument to the training set of the first. This is useful when trying to load data from multiple stations, but testing on one station only.
* `binarize(data, target)` - Given output of `load()` and either 1 or -1, binarize the target variable to 0 or 1. Whatever class is in the second argument will become '1' in the new data.

**The `run_logreg` package**

We wrote this python package so running many logistic regression models with different parameters is cleaner and easier. Here are the methods included:

* `do_logreg` - Takes the result of split_data and performs a logistic regression given the input parameters. It takes a parameters called "squares" that will perform some basic feature engineering by squaring some of the variables, a penalty function (l1 or l2, defaults to l2), and a c parameter (defaults to 100,000).

* `distance` - Calculates the distance betweent two stations, based on the distance formula

* `closest_stations` - Identifies the stations closest to the input station, using the `distance` function

* `add_closest_stations` - Takes split_data for one station and its station_id, splits, merges the closest stations' data and binarizes all

* `format_plot` - Formats the plot according to the desired target_recall and whether this is a plot of empty or full

### Logistic Regression Model

In [2]:
class Logistic_Regression_Specs():
    def __init__(self, split_data, stationid, target, empty=True, squares=False, num_append=0, C=1e5, penalty='l2'):
        self.stationid = stationid
        self.target = target
        self.split_data = split_data
        self.empty = empty
        self.squares = squares
        self.num_append = num_append
        self.penalty = penalty
        self.C = C

def construct_key(spec):
    key = ''
    if spec.target != 'y_60m':
        key = key + spec.target + ' '
    if spec.squares == True:
        key = key + 'squares; '
    if spec.num_append > 0:
        key = key + 'append: ' + str(spec.num_append) + '; '
    key = key + 'penalty: ' + spec.penalty + '; '
    key = key + 'c: ' + str(spec.C) + '; '
    return key

def run_models(list_of_specs):
    '''Creates a dictionary of models based on list of specs objects'''
    
    logregs = {}
    scalers = {}
    predictions = {}
    specs = {}
    
    for spec in list_of_specs:
        logregs[construct_key(spec)], scalers[construct_key(spec)], predictions[construct_key(spec)] = run_logreg.do_logreg(spec, plot = False) 
        specs[(construct_key(spec))] = spec
    return logregs, scalers, predictions, specs

def pr_curve(predictions, true_value, target_recall=0.95):
    curve = precision_recall_curve(true_value, predictions)
    precision, recall, thresholds = curve
    mp, mr, mt = project_env.max_precision_for_recall(curve, target_recall=target_recall)
    return mp, mr, mt

In [3]:
# test loading data
data = project_env.load_split_bucket(519, target='y_60m', log=False)
print('done loading')


# merge test
train_X, train_y = data['train']
dev_X, dev_y = data['dev']
test_X, test_y = data['test']

print(train_X.shape)
print(train_y.shape)
print(dev_X.shape)
print(test_X.shape)

merged_X = pd.concat([train_X, dev_X])
merged_y = pd.concat([train_y, dev_y])
print(merged_X.shape)
print(merged_y.shape)


done loading
(4236, 22)
(4236,)
(968, 22)
(968, 22)
(5204, 22)
(5204,)


In [4]:
# best prediction: ['y_60m', False, 0, 10, 'l1']

data = project_env.load_split_bucket(519, target='y_60m', log=False)
spec = Logistic_Regression_Specs(data, stationid=519, target='y_60m', empty=True, squares=False, num_append=0, C=1.0, penalty='l1')

logregs_e, scalers_e, predictions_e = run_logreg.do_logreg(spec, plot=False)

logregs_e, scalers_e, predictions_e = run_logreg.do_logreg(spec, plot=False, merge_train_dev=True)

#data_empty = project_env.binarize(data, -1)
#gold_labels = data_empty['dev'][1]


Training set X shape: (4236, 22)
Trained on train set of 4236 examples
Evaluating on dev set of 968 examples
Accuracy: 0.813016528926
[[448 137]
 [ 44 339]]
Training set X shape: (5204, 22)
Trained on train set of 5204 examples
Evaluating on dev set of 968 examples
Accuracy: 0.785123966942
[[632 169]
 [ 39 128]]


In [5]:
#target_vars = ['y_10m','y_15m','y_30m','y_45m','y_60m','y_90m','y_120m']
target_vars = ['y_30m', 'y_60m']
c_list = [.01, .1, 1, 10, 100, 1000, 10000]
#c_list = [.01, 1, 100]
penalties = ['l1']
#sq_list = [True, False]
sq_list = [False]
#num_append_list = [0,1,10]
num_append_list = [0]
target_list = {}

df_dict = {}

#Returns a dictionary within a dictionary.  Outer dictionary is keyed on target var. inner dictionary keyed on a 
# "model number" and contains the specs of the model as well as a list of the [mp,mr,mt]
# You can probably used this to find the max MR for each target variable
for target in target_vars:
    data = project_env.load_split_bucket(519, target=target, log=False)
    data_empty = project_env.binarize(data, -1)
    # gold_labels = data_empty['dev'][1]
    gold_labels_e = data_empty['test'][1]
    data_full = project_env.binarize(data, 1)
    gold_labels_f = data_full['test'][1]
    
    models = {}
    model_id=0
    rows = int(len(sq_list))*int(len(num_append_list))*int(len(c_list))*int(len(penalties))*2
    
    columns=['specs', 'mp', 'mr', 'mt']
    df_dict[target] = pd.DataFrame(data=np.zeros((rows,len(columns))), columns=columns) 
    
    for squares in sq_list:
        for num_append in num_append_list:
            for c in c_list:
                for penalty in penalties:

                    print([target, squares, num_append, c, penalty])
                    
                    spec_e = Logistic_Regression_Specs(data, 519, target, empty=True, squares=squares, num_append=num_append, C=c, penalty=penalty)
                    spec_f = Logistic_Regression_Specs(data, 519, target, empty=False, squares=squares, num_append=num_append, C=c, penalty=penalty)
                    
                    #logregs_e, scalers_e, predictions_e = run_logreg.do_logreg(spec, plot=False)
                    logregs_e, scalers_e, predictions_e = run_logreg.do_logreg(spec_e, plot=False, merge_train_dev=True)
                    logregs_f, scalers_f, predictions_f = run_logreg.do_logreg(spec_f, plot=False, merge_train_dev=True)
                    #print(predictions_e)
                    
                    mp, mr, mt = pr_curve(predictions_e, gold_labels_e, target_recall=0.95)
                    df_dict[target]['specs'].loc[model_id] = ['empty', target, squares, num_append, c, penalty]
                    df_dict[target]['mp'].loc[model_id] = mp
                    df_dict[target]['mr'].loc[model_id] = mr
                    df_dict[target]['mt'].loc[model_id] = mt
                    #model_specs = [target,squares,num_append,c,penalty]
                    #models.setdefault(model_id,[]).append([model_specs,mp,mr,mt])
                    model_id = model_id + 1
                    
                    mp, mr, mt = pr_curve(predictions_f, gold_labels_f, target_recall=0.95)
                    df_dict[target]['specs'].loc[model_id] = ['full', target, squares, num_append, c, penalty]
                    df_dict[target]['mp'].loc[model_id] = mp
                    df_dict[target]['mr'].loc[model_id] = mr
                    df_dict[target]['mt'].loc[model_id] = mt
                    model_id = model_id + 1

    

['y_30m', False, 0, 0.01, 'l1']
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.890927624873
[[744  71]
 [ 36 130]]
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.900101936799
[[883   0]
 [ 98   0]]


/Users/id460/miniconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


['y_30m', False, 0, 0.1, 'l1']
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.870540265036
[[728  87]
 [ 40 126]]
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.90621814475
[[872  11]
 [ 81  17]]
['y_30m', False, 0, 1, 'l1']
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.86748216106
[[711 104]
 [ 26 140]]
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.900101936799
[[873  10]
 [ 88  10]]
['y_30m', False, 0, 10, 'l1']
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.863404689093
[[705 110]
 [ 24 142]]
Training set X shape: (5286, 22)
Trained on train set of 5286 examples
Evaluating on dev set of 981 examples
Accuracy: 0.904179

In [6]:
# optional save as CSV
#for target in target_vars:
#    df_dict[target].to_csv(target + ".merged_train_dev.csv")

# output:
# [target,squares,num_append,c,penalty]
# max_pre, max_rec, max_thresh

### optimal Cs based on max prediction:

 - y_30m merged_train_dev empty - 0.01
 - y_30m merged_train_dev full - 0.1
 - y_60m merged_train_dev empty - 100
 - y_60m merged_train_dev full - 0.1

In [55]:
from os import listdir
from os.path import isfile, join
import string

data_path = 'per_station_train'
station_files = sorted([f for f in listdir(data_path) if isfile(join(data_path, f))])
station_files = list(filter(lambda f: 'swp' not in f, station_files))
print('total station files: ' + str(len(station_files)))
print('head files: ' + str(station_files[:5]))

total station files: 49
head files: ['128.csv', '151.csv', '161.csv', '174.csv', '229.csv']


In [56]:
settings_columns = ['target', 'c']

settings_e = dict()
settings_e['y_10m'] = 0.01;
settings_e['y_15m'] = 0.01;
settings_e['y_30m'] = 0.01;
settings_e['y_45m'] = 0.01;
settings_e['y_60m'] = 1000;
settings_e['y_90m'] = 0.1;
settings_e['y_120m'] = 1;

settings_f = dict()
settings_f['y_10m'] = 0.01;
settings_f['y_15m'] = 0.01;
settings_f['y_30m'] = 0.1;
settings_f['y_45m'] = 10;
settings_f['y_60m'] = 10;
settings_f['y_90m'] = 1000;
settings_f['y_120m'] = 1000;


In [57]:
model_id = 0

columns=['station_id', 'status', 'target', 'c', 'mp', 'mr', 'mt']
multi_sample_res = pd.DataFrame(columns=columns)

# some stations cause errors ("Found input variables with inconsistent numbers of samples")
bad_stations = ['3255', '379', '442', '446', '450', '457', '477', '479', '490', '492', 
                '494', '505', '523']

target_vars = ['y_10m','y_15m','y_30m','y_45m','y_60m','y_90m','y_120m']
#target_vars = ['y_30m', 'y_60m']

for s in station_files:
    station_id = s.replace('.csv', '')
    print('station: ' + station_id)
    
    # check if current station is one of the bad ones
    if station_id in bad_stations:
        print('skip bad station:' + station_id)
        continue
    
    for target in target_vars:
        data = project_env.load_split_bucket(station_id, target=target, log=False)
        data_empty = project_env.binarize(data, -1)
        gold_labels_e = data_empty['test'][1]
        data_full = project_env.binarize(data, 1)
        gold_labels_f = data_full['test'][1]
        

        c_val_e = settings_e[target]
        spec_e = Logistic_Regression_Specs(data, station_id, target, empty=True, squares=False, num_append=0, C=c_val_e, penalty='l1')
        c_val_f = settings_f[target]
        spec_f = Logistic_Regression_Specs(data, station_id, target, empty=False, squares=False, num_append=0, C=c_val_f, penalty='l1')

        logregs_e, scalers_e, predictions_e = run_logreg.do_logreg(spec_e, plot=False, merge_train_dev=True)
        logregs_f, scalers_f, predictions_f = run_logreg.do_logreg(spec_f, plot=False, merge_train_dev=True)

        mp, mr, mt = ['err', 'err', 'err']

        mp, mr, mt = pr_curve(predictions_e, gold_labels_e, target_recall=0.95)
        multi_sample_res.loc[model_id] = [station_id, 'empty', target, c_val_e, mp, mr, mt]
        model_id = model_id + 1

        mp, mr, mt = pr_curve(predictions_f, gold_labels_f, target_recall=0.95)
        multi_sample_res.loc[model_id] = [station_id, 'full', target, c_val_f, mp, mr, mt]
        model_id = model_id + 1


station: 128
Training set X shape: (3921, 22)
Trained on train set of 3921 examples
Evaluating on dev set of 566 examples
Accuracy: 0.982332155477
[[556   0]
 [ 10   0]]
Training set X shape: (3921, 22)
Trained on train set of 3921 examples
Evaluating on dev set of 566 examples
Accuracy: 0.885159010601
[[402  55]
 [ 10  99]]
Training set X shape: (3935, 22)
Trained on train set of 3935 examples
Evaluating on dev set of 578 examples
Accuracy: 0.982698961938
[[568   0]
 [ 10   0]]
Training set X shape: (3935, 22)
Trained on train set of 3935 examples
Evaluating on dev set of 578 examples
Accuracy: 0.870242214533
[[406  59]
 [ 16  97]]
Training set X shape: (3863, 22)
Trained on train set of 3863 examples
Evaluating on dev set of 548 examples
Accuracy: 0.981751824818
[[538   0]
 [ 10   0]]
Training set X shape: (3863, 22)
Trained on train set of 3863 examples
Evaluating on dev set of 548 examples
Accuracy: 0.802919708029
[[373  65]
 [ 43  67]]
Training set X shape: (3843, 22)
Trained on t

In [59]:
multi_sample_res.to_csv('y.multi-station.csv')
multi_sample_res.head()


,station_id,status,target,c,mp,mr,mt
0,128,empty,y_10m,0.01,0.277778,1.000000,0.160619
1,128,full,y_10m,0.01,0.447257,0.972477,0.379214
2,128,empty,y_15m,0.01,0.357143,1.000000,0.174237
3,128,full,y_15m,0.01,0.426877,0.955752,0.336271
4,128,empty,y_30m,0.01,0.192308,1.000000,0.139518
